In [ ]:
import pandas as pd
import numpy as np
test_base = pd.read_csv('/data/testa_base.csv')
test_trx = pd.read_csv('/data/testa_trx.csv')
print(test_base.shape, test_trx.shape)

In [ ]:
test_trx.head()

In [ ]:
# 统计每个客户的交易次数和交易金额
df_count = test_trx.groupby("cust_wid").agg({"trx_amt": ["count", "sum", "min", "max", "mean"]})
df_count.columns = ["trx_count", "trx_sum", "trx_min", "trx_max", "trx_mean"]
df_count =  df_count.reset_index()
df_count.head()

In [ ]:
df_unique = test_trx.groupby("cust_wid").nunique()
df_unique.columns = ["trx_cd_unique", "trx_amt_unique", "trx_tm_unique"]
df_unique =  df_unique.reset_index()
df_unique.head()

In [ ]:
import datetime as dt
test_trx['trx_tm'] = test_trx['trx_tm'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') if type(x)==str else pd.NaT)
test_trx['trx_tm']

In [ ]:
test_trx["year"] = test_trx["trx_tm"].apply(lambda x: x.year)
test_trx["month"] = test_trx["trx_tm"].apply(lambda x: x.month)
test_trx["day"] = test_trx["trx_tm"].apply(lambda x: x.day)
test_trx["hour"] = test_trx["trx_tm"].apply(lambda x: x.hour)

test_trx.head()

In [ ]:
active_time = test_trx.groupby("cust_wid")["trx_tm"].agg(["min", "max"])
active_time.columns = ["first_trx_time", "last_trx_time"]

In [ ]:
active_time["active_time"] = active_time["last_trx_time"] - active_time["first_trx_time"]
active_time["active_time_days"] = active_time["active_time"].astype("timedelta64[D]")
active_time

In [ ]:
test_trx.sort_values(by=['cust_wid', 'trx_tm'], inplace=True)

# 计算每个用户登陆的日期列表，并按照日期顺序排序
date_series = test_trx['trx_tm'].apply(lambda x: x.date())
login_dates = test_trx.groupby('cust_wid')['trx_tm'].apply(lambda x: sorted(set(x.apply(lambda y: y.date()))))

# 计算每个用户最长连续登陆天数
max_consecutive_days = {}
for cust_wid, dates in login_dates.iteritems():
    max_consecutive_days[cust_wid] = 1
    consecutive_days = 1
    for i in range(1, len(dates)):
        if (dates[i] - dates[i-1]).days == 1:
            consecutive_days += 1
            max_consecutive_days[cust_wid] = max(max_consecutive_days[cust_wid], consecutive_days)
        else:
            consecutive_days = 1

max_consec = pd.DataFrame(list(max_consecutive_days.items()), columns=['cust_wid', 'max_consec'])


In [ ]:
print(df_count.shape, df_unique.shape, active_time.shape, max_consec.shape)
final_out = pd.merge(df_count,df_unique, on = ['cust_wid'])
print(final_out.shape)
final_out = pd.merge(final_out,active_time.reset_index()[['cust_wid','active_time_days']], on = ['cust_wid'])
print(final_out.shape)
final_out = pd.merge(final_out,max_consec, on = ['cust_wid'])
print(final_out.shape)
final_out.head()

In [ ]:
print(final_out.shape)
final_out.to_csv('./test_fea1.csv',index=0)

In [ ]:
(0.7692307692307692*50000)/(5-4*0.7692307692307692)